In [2]:
import numpy as np
import csv
import cv2

import math
from PIL import Image
from torchvision.transforms import PILToTensor,ToTensor,Normalize,Resize
from torchvision import transforms
import torch
import torch
import motmetrics
import random
import tqdm
import os
from torchreid.utils import FeatureExtractor
from torch.utils.data import Dataset, DataLoader

import torchreid
from data_utils.data_load import simple_dataset
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import  seaborn as sns
from sacred import Experiment

In [13]:
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=20,
    pretrained=True
)


Successfully loaded imagenet pretrained weights from "/home/gauthier/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [15]:
from torch import nn

model.classifier= nn.Linear(in_features=model.classifier.in_features, out_features=500)
model

OSNet(
  (conv1): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): OSBlock(
      (conv1): Conv1x1(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2a): LightConv3x3(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv2b): Sequential(
        (

In [ ]:
##to run you will have to run gen_crops with your path 

In [ ]:



seed = 10

random.seed(seed)

torch.manual_seed(seed)

np.random.seed(seed)


In [ ]:
data_mot = '1'

In [ ]:
##change this to your path
dataset_dir = '/media/data/gauthier/mot_cropped/'+data_mot+'/'
all_crops = os.listdir(dataset_dir)
tuple_list = []
##tuple = [data,label]
for crop in all_crops:
    tuple_list += [(dataset_dir+crop,int(crop.split('_')[0]))]

len(tuple_list)

In [ ]:
#ex = Experiment('tsne',interactive=True)
#ex.add_config('/home/gauthier/reid/res50-mot17-batch_hard/sacred_config.yaml')


In [ ]:

cnn = FeatureExtractor(
    model_name='osnet_x1_0',
   model_path='../model_saves/cnn_triplet.pth.tar',#'../model_saves/cnn_softmax.pth.tar'
    device='cuda'
)


In [ ]:
dataset = simple_dataset(tuple_list)

In [ ]:



dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=5, drop_last=True)


In [ ]:

transform = transforms.Compose([
     transforms.Resize((256, 128)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
])

 
labels =[]
outputs =[]
images= []

for i,batch in enumerate(dataloader):
    with torch.no_grad():
        images_path = batch[0]
        lab_batch = [int(b) for b in batch[1] ]
        labels += lab_batch

        images += images_path
        images_pil = [Image.open(img_path) for img_path in images_path]


        inputs = torch.stack([transform(img_pil) for img_pil in images_pil])

        output = cnn(inputs.to('cuda'))



        current_outputs = output.cpu().numpy()
        outputs.append(current_outputs)
    #features = np.concatenate((outputs, current_outputs))


In [ ]:
features = np.stack(outputs,axis=0)

In [ ]:
features = features.reshape(-1, features.shape[-1])

In [ ]:
tsne = TSNE(n_components=2).fit_transform(features)


In [ ]:

def scale_to_01_range(x):

    # compute the distribution range

    value_range = (np.max(x) - np.min(x))
    # move the distribution so that it starts from zero

    # by extracting the minimal value from all its values

    starts_from_zero = x - np.min(x)

 

    # make the distribution fit [0; 1] by dividing by its range

    return starts_from_zero / value_range



tx = tsne[:, 0]
ty = tsne[:, 1]

 

tx = scale_to_01_range(tx)
ty = scale_to_01_range(ty)


In [ ]:
tuple_np = np.array(tuple_list)
num_classes = np.unique(tuple_np[:,1])
num_classes = np.asarray(num_classes,dtype=int)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)
 

# for every class, we'll add a scatter plot separately
colors_per_class = { label : sns.color_palette("hls", num_classes.shape[0])[label] for label in num_classes }
for label in colors_per_class:

    # find the samples of the current class in the data
    
    indices = [i for i, l in enumerate(labels) if l == label]
    # extract the coordinates of the points of this class only

    current_tx = np.take(tx, indices)

    current_ty = np.take(ty, indices)
    # convert the class color to matplotlib format
    color = np.array(colors_per_class[label], dtype=np.float)
 
    # add a scatter plot with the corresponding color and label
    ax.scatter(current_tx, current_ty, color=color, label=label)
 
ax.legend(loc='best')
# finally, show the plot
plt.show()


In [ ]:
def scale_image(image, max_image_size):
    image_height, image_width, _ = image.shape

    scale = max(1, image_width / max_image_size, image_height / max_image_size)
    image_width = int(image_width / scale)
    image_height = int(image_height / scale)

    image = cv2.resize(image, (image_width, image_height))
    return image

def draw_rectangle_by_class(image, label):
    image_height, image_width, _ = image.shape

    # get the color corresponding to image class
    color = colors_per_class[label]
    image = cv2.rectangle(image, (0, 0), (image_width - 1, image_height - 1), color=color, thickness=5)

    return image


In [ ]:
plot_size =2000
max_image_size =100
offset = max_image_size // 2

image_centers_area_size = plot_size - 2 * offset


def compute_plot_coordinates(image, x, y, image_centers_area_size, offset):

    image_height, image_width, _ = image.shape

 

    # compute the image center coordinates on the plot

    center_x = int(image_centers_area_size * x) + offset
 
    # in matplotlib, the y axis is directed upward
    # to have the same here, we need to mirror the y coordinate
    center_y = int(image_centers_area_size * (1 - y)) + offset
 
    # knowing the image center,
    # compute the coordinates of the top left and bottom right corner
    tl_x = center_x - int(image_width / 2)
    tl_y = center_y - int(image_height / 2)
    br_x = tl_x + image_width
    br_y = tl_y + image_height
 
    return tl_x, tl_y, br_x, br_y
 
# we'll put the image centers in the central area of the plot
# and use offsets to make sure the images fit the plot
 
# init the plot as white canvas
tsne_plot = 255 * np.ones((plot_size, plot_size, 3), np.uint8)
 
# now we'll put a small copy of every image to its corresponding T-SNE coordinate
for image_path, label, x, y in zip(images, labels, tx, ty):
        
    image = cv2.imread(image_path)
 
    # scale the image to put it to the plot
    image = scale_image(image, max_image_size)
 
    # draw a rectangle with a color corresponding to the image class
    image = draw_rectangle_by_class(image, label)
 
    # compute the coordinates of the image on the scaled plot visualization
    tl_x, tl_y, br_x, br_y = compute_plot_coordinates(image, x, y, image_centers_area_size, offset)

    # put the image to its t-SNE coordinates using numpy sub-array indices
    tsne_plot[tl_y:br_y, tl_x:br_x, :] = image
    cv2.putText(tsne_plot,str(label), (tl_x,tl_y), 0, 2, 255)

cv2.imwrite('t_sne.png', tsne_plot)
#imshow will crashs the notebook

In [23]:
import urllib
import torch
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)


In [24]:
from PIL import Image
from torchvision import transforms
import torch
from torch import nn
import torch.nn.functional as F
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)
model = nn.Sequential(*list(model.children())[:-2])

model.eval()

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0].shape)
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.


Using cache found in /home/gauthier/.cache/torch/hub/pytorch_vision_v0.6.0


torch.Size([2048, 7, 7])


In [25]:
otp = output[0]

In [26]:
#otp = torch.reshape(otp,(49,2048))

In [114]:
import torch.nn.functional as f
import math
class Conv1dSame(nn.Module):
    """Represents the "Same" padding functionality from Tensorflow.
    NOTE: Only work correctly when dilation == 1, groups == 1 !!!
    https://github.com/pytorch/pytorch/issues/3867
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super().__init__()
        self.cut_last_element = (kernel_size % 2 == 0 and stride == 1 and dilation % 2 == 1)
        self.padding = math.ceil((1 - stride + dilation * (kernel_size-1))/2)
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=self.padding, stride=stride, dilation=dilation)

    def forward(self, x):
        if self.cut_last_element:
            return self.conv(x)[:, :, :-1]
        else:
            return self.conv(x)
        
class Conv1dPaddingSame(nn.Module):
    '''pytorch version of padding=='same'
    ============== ATTENTION ================
    Only work when dilation == 1, groups == 1
    =========================================
    '''
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(Conv1dPaddingSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.weight = nn.Parameter(torch.rand((out_channels, 
                                                 in_channels, kernel_size)))
        # nn.Conv1d default set bias=True，so create this param
        self.bias = nn.Parameter(torch.rand(out_channels))
        
    def forward(self, x):
        batch_size, num_channels, length = x.shape
        if length % self.stride == 0:
            out_length = length // self.stride
        else:
            out_length = length // self.stride + 1

        pad = math.ceil((out_length * self.stride + 
                         self.kernel_size - length - self.stride) / 2)
        out = F.conv1d(input=x, 
                       weight = self.weight,
                       stride = self.stride, 
                       bias = self.bias,
                       padding=pad)
        return out


class SpatialAttention(torch.nn.Module):
    def __init__(self,is_train ):
        super(SpatialAttention, self).__init__()        
        resnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
        self.conv_features = nn.Sequential(*list(resnet.children())[:-2])
        for param in self.conv_features.parameters():
            param.requires_grad = True
        
        self.conv1d = Conv1dPaddingSame(49,1,1,1)
        self.softmax = nn.Softmax(dim=-1)
        
        self.last_layer = nn.Linear(in_features=2*2048,out_features=512)
        self.train_layer = nn.Linear(in_features=512,out_features=1)
        self.is_train = is_train
    def forward(self, old_feature, new_feature):
        
        
        new_conv = self.conv_features(new_feature)
        old_conv = self.conv_features(old_feature)
        
        new_conv = new_conv.permute(0,2,3,1)
        new_conv = torch.reshape(new_conv,(new_conv.shape[0],49,2048))

        old_conv = old_conv.permute(0,2,3,1)
        old_conv = torch.reshape(old_conv,(old_conv.shape[0],49,2048))
        
        
        
        norm_old_conv = f.normalize(old_conv,dim=0,p=-1)
        norm_new_conv = f.normalize(new_conv,dim=0,p=-1)
        norm_new_conv_t = new_conv.permute(0,2,1)
        
        new_match = torch.bmm(norm_old_conv,norm_new_conv_t)
        

        old_match = new_match.permute(0,2,1)
        
        new_conv1d = self.conv1d(new_match)
        old_conv1d = self.conv1d(old_match)
            
        
        new_att = torch.reshape(new_conv1d,(new_conv1d.shape[0],49))
        old_att = torch.reshape(new_conv1d,(new_conv1d.shape[0],49))
        
        new_att = self.softmax(new_att)
        old_att = self.softmax(old_att)
        
        new_att = torch.reshape(new_conv1d,(new_conv1d.shape[0],49,1))
        old_att = torch.reshape(new_conv1d,(new_conv1d.shape[0],49,1))

        #new_conv_atted = torch.bmm(new_conv,new_att)
        #old_conv_atted = torch.bmm(old_conv,old_att)
        new_conv_atted = new_conv * new_att
        old_conv_atted = old_conv * old_att

        new_conv_atted_sum = torch.sum(new_conv_atted, dim=1)
        old_conv_atted_sum = torch.sum(old_conv_atted, dim=1)
        
        catted = torch.cat([new_conv_atted_sum,old_conv_atted_sum],dim=1)
        
        out_feature =  F.relu(self.last_layer(catted))
        
        if self.is_train:
            bin_class = self.train_layer(out_feature)
            return bin_class,new_conv_atted_sum,new_conv_atted_sum
        return out_feature



In [115]:
e = SpatialAttention()

Using cache found in /home/gauthier/.cache/torch/hub/pytorch_vision_v0.6.0


In [116]:
e(input_batch,input_batch)

torch.Size([1, 49, 49])
torch.Size([1, 1, 49])
torch.Size([1, 49])
tensor([[20321.7949, 18147.2051, 18455.6406, 13869.2510,  4315.6860,  6013.4707,
          6439.5020, 19580.2598, 18510.7715, 21936.9277, 20676.4414,  6510.4326,
          6808.3188,  7364.6782, 23788.3262, 23139.1699, 23391.9707, 24493.1055,
         14593.8633,  9144.5596,  8480.2109, 19389.6621, 19129.3008, 17385.1699,
         17292.3340, 13165.7002,  7942.8003,  7376.9873, 15499.4795, 10211.5156,
          8024.8042,  7639.8252, 10640.0000, 10206.1104, 13503.0732, 18574.6523,
         13842.2520, 10002.1230,  6917.8418, 13336.1738, 14061.9150, 20125.8945,
         14981.3018, 11414.4121,  9760.4688,  6178.6562, 13524.9785, 15691.7314,
         20648.0879]], grad_fn=<ViewBackward>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       grad_f

NameError: name 'out' is not defined